In [95]:
import pickle
import numpy as np

In [96]:
with open('embedded_vectors.pkl', 'rb') as f:
    embedded_vectors = pickle.load(f)

In [97]:
with open("output11.txt") as file:
    corpus = file.read()

In [98]:
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\OMEN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [99]:
sentences = sent_tokenize(corpus)

In [100]:
document = [{'sentence': x, 'index' : i} for i, x in enumerate(sentences)]

In [101]:
def combine_sentences(sentences, buffer_size=1):
    # Go through each sentence dict
    for i in range(len(sentences)):

        # Create a string that will hold the sentences which are joined
        combined_sentence = ''

        # Add sentences before the current one, based on the buffer size.
        for j in range(i - buffer_size, i):
            # Check if the index j is not negative (to avoid index out of range like on the first one)
            if j >= 0:
                # Add the sentence at index j to the combined_sentence string
                combined_sentence += sentences[j]['sentence'] + ' '

        # Add the current sentence
        combined_sentence += sentences[i]['sentence']

        # Add sentences after the current one, based on the buffer size
        for j in range(i + 1, i + 1 + buffer_size):
            # Check if the index j is within the range of the sentences list
            if j < len(sentences):
                # Add the sentence at index j to the combined_sentence string
                combined_sentence += ' ' + sentences[j]['sentence']

        # Then add the whole thing to your dict
        # Store the combined sentence in the current sentence dict
        sentences[i]['combined_sentence'] = combined_sentence

    return sentences

In [102]:
sentences = combine_sentences(document)

In [103]:
com_sentences = []

for x in sentences:
    com_sentences.append(x['combined_sentence'])

In [104]:
indexes123 = np.load('indexes.npy')

In [105]:
indices_above_thresh = indexes123

In [106]:
# Initialize the start index
start_index = 0

# Create a list to hold the grouped sentences
chunks = []

# Iterate through the breakpoints to slice the sentences
for index in indices_above_thresh:
    # The end index is the current breakpoint
    end_index = index

    # Slice the sentence_dicts from the current start index to the end index
    group = sentences[start_index:end_index + 1]
    combined_text = ' '.join([d['sentence'] for d in group])
    chunks.append(combined_text)
    
    # Update the start index for the next group
    start_index = index + 1

# The last group, if any sentences remain
if start_index < len(sentences):
    combined_text = ' '.join([d['sentence'] for d in sentences[start_index:]])
    chunks.append(combined_text)


In [107]:
embedded_vectors = embedded_vectors.tolist()

In [108]:
for i in range(len(chunks)-1, -1, -1):
    if len(chunks[i]) > 10000:
        chunks.pop(i)
        embedded_vectors.pop(i)


In [109]:
from pymilvus import (
    connections,
    utility,
    FieldSchema,
    CollectionSchema,
    DataType,
    Collection,
)

In [110]:
connections.connect("default", host="localhost", port="19530")

In [111]:
chunks[25]

'// 4. Compute saxpy and write back to shared memory\nfor (int i = threadIdx.x; i < buf_len; i += blockDim.x) {\n// 5. Wait for shared memory writes to be visible to TMA engine. // After syncthreads, writes by all threads are visible to TMA engine. // 6. Initiate TMA transfer to copy shared memory to global memory\n// 7. Wait for TMA transfer to have finished reading shared memory. // Create a "bulk async-group" out of the previous bulk copy operation. // Wait for the group to have completed reading from shared memory. Barrier initialization. The barrier is initialized with the number of\nthreads participating in the block. As a result, the barrier will flip only if\nall threads have arrived on this barrier. Shared memory barriers are described\nin more detail in Asynchronous Data Copies using cuda::barrier. To make the initialized barrier visible to subsequent bulk-asynchronous copies, the\nfence.proxy.async.shared::cta instruction is used. This instruction ensures that\nsubsequent bu

In [127]:
import grpc
from concurrent import futures

# Create a gRPC server with increased max message length
server = grpc.server(
    futures.ThreadPoolExecutor(max_workers=10),
    options=[('grpc.max_send_message_length', 100 * 1024 * 1024), 
             ('grpc.max_receive_message_length', 100 * 1024 * 1024)]
)

In [112]:
from pymilvus import Collection, CollectionSchema, FieldSchema, DataType

# Define the schema
fields = [
    FieldSchema(name="SrNo", dtype=DataType.INT64, is_primary=True, auto_id=True),
    FieldSchema(name="combined_chunks", dtype=DataType.VARCHAR,max_length=10000),
    FieldSchema(name="embeddings", dtype=DataType.FLOAT_VECTOR, dim=384),
]

schema = CollectionSchema(fields, "Nvidia_CUDA_Chunked_embeddings")

# Create the collection
collection = Collection("N_cuda", schema)


In [113]:
sr_nos = [None] * 40763

In [114]:
len(embedded_vectors[1])

384

In [115]:
type(chunks)

list

In [116]:
data = [chunks,embedded_vectors]

In [123]:
batch_size = 100

In [ ]:
for start in range(0, len(chunks), batch_size):
    end = min(start + batch_size, len(chunks))
    batch_combined_chunks = chunks[start:end]
    batch_embeddings = embedded_vectors[start:end]
    data = [chunks, embedded_vectors]
    collection.insert(data)

In [ ]:
client = Milvus(host='localhost', port='19530')

In [129]:
collection_name = 'N_cuda'

# Load the collection
collection = Collection(collection_name)